In [2]:
import pandas as pd
import numpy as np

def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()

df = pd.read_csv('availability.csv')

df.rename(columns={'last_update': 'time_of_day'}, inplace=True)
df = df.drop(columns=["status"])
df['time_of_day'] = pd.to_datetime(df['time_of_day'])

specified_date_time = pd.to_datetime('2024-02-25 15:03:04')

df = df[df['time_of_day'] >= specified_date_time]
df2 = pd.read_csv('weather.csv')
df2['time_of_day'] = pd.to_datetime(df2['time_of_day'])
df2 = df2.drop(columns=["main"])

merged_df = pd.merge_asof(df.sort_values('time_of_day'), df2.sort_values('time_of_day'), 
                          by='station_id', on='time_of_day', direction='nearest')

merged_df['wind_speed'] = merged_df['wind_speed'].interpolate(method='linear')
merged_df['temperature'] = merged_df['temperature'].interpolate(method='linear')
df = merged_df
df


,station_id,time_of_day,available_bike_stands,bike_stands,available_bikes,description,temperature,wind_speed
0,42,2024-02-25 15:03:04,0,30,30,broken clouds,7,9.77
1,51,2024-02-25 15:03:06,34,40,6,broken clouds,8,9.77
2,32,2024-02-25 15:03:06,0,30,30,broken clouds,8,9.77
3,18,2024-02-25 15:03:07,6,30,24,broken clouds,7,9.77
4,57,2024-02-25 15:03:08,23,23,0,broken clouds,7,9.77
...,...,...,...,...,...,...,...,...
490185,28,2024-03-20 22:49:29,26,30,4,clear sky,6,4.12
490186,108,2024-03-20 22:49:33,27,35,8,clear sky,5,4.12
490187,56,2024-03-20 22:49:34,27,40,13,clear sky,6,4.12
490188,90,2024-03-20 22:49:37,37,40,3,clear sky,6,4.12


In [5]:
df = df.rename(columns={'time_of_day': 'ds', 'available_bikes': 'y'})

In [6]:
train = df.loc[df['ds'] < '2024-03-05']
valid = df.loc[(df['ds'] >= '2024-03-05') & (df['ds'] < '2024-03-08')]

In [10]:
from gluonts.dataset.pandas import PandasDataset

train_ds = PandasDataset.from_long_dataframe(train, target='available_bikes', item_id='station_id', 
                                       timestamp='ds', freq='m')

In [24]:
from gluonts.torch.model.deepar import DeepAREstimator

estimator = DeepAREstimator(freq='D', prediction_length=90, num_layers=3, trainer_kwargs={'accelerator': 'gpu', 'max_epochs':30})

predictor = estimator.train(train_ds, num_workers=2)

ModuleNotFoundError: No module named 'lightning'